<a href="https://colab.research.google.com/github/somilasthana/MachineLearningSkills/blob/master/NLP_Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fastai

In [0]:
from fastai import *
from fastai.text import *
import pandas as pd

In [0]:
import sklearn.feature_extraction.text as sklearn_text

In [0]:
?? URLs

In [0]:
#path = untar_data("http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz")
path = untar_data(URLs.IMDB_SAMPLE)

In [6]:
path

PosixPath('/root/.fastai/data/imdb_sample')

In [7]:
!ls /root/.fastai/data/imdb_sample

texts.csv


In [0]:
df = pd.read_csv("/root/.fastai/data/imdb_sample/texts.csv")

In [9]:
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [0]:
movie_reviews = TextList.from_csv(path, 'texts.csv', cols='text').split_from_df(col=2).label_from_df(cols=0)

In [11]:
movie_reviews.train.x[10]

Text xxbos xxmaj this movie is the first of xxmaj xxunk triad society trilogy , and the trilogy kicks of to a great start . xxmaj the movies in the trilogy are only connected xxunk , and these themes are actually apparent in all his films , if you look close enough . xxmaj xxunk xxmaj triad xxmaj society is about a cop trying to prevent his kid brother from getting too involved with a rather extreme gang of outsiders , struggling their way to the top of xxmaj xxunk xxunk . xxmaj the kid brother is a lawyer , and the triad gang is becoming increasingly in need of one , as the movie xxunk . xxmaj the movie takes place in a very harsh environment , and is therefore pretty violent and tough . xxmaj xxunk has done worse , but since this is a serious movie it hits you very hard . xxmaj as usual there is also a lot of xxunk sex , mostly homosexual in this one . xxmaj the movie is in many ways a typical gangster movie , but with a great drive and true xxunk . xxmaj if you 've only seen xxmaj x

In [12]:
movie_reviews.train.y[10]

Category positive

In [13]:
df.iloc[10]["text"]

"This movie is the first of Miikes triad society trilogy, and the trilogy kicks of to a great start. The movies in the trilogy are only connected thematically, and these themes are actually apparent in all his films, if you look close enough. Shinjuku Triad Society is about a cop trying to prevent his kid brother from getting too involved with a rather extreme gang of outsiders, struggling their way to the top of Tokyos yakuza. The kid brother is a lawyer, and the triad gang is becoming increasingly in need of one, as the movie progresses. The movie takes place in a very harsh environment, and is therefore pretty violent and tough. Miike has done worse, but since this is a serious movie it hits you very hard. As usual there is also a lot of perverted sex, mostly homosexual in this one. The movie is in many ways a typical gangster movie, but with a great drive and true grittiness. If you've only seen Miikes far-out movies (Ichi the killer, Fudoh etc.) this is worth checking out since it

In [14]:
movie_reviews.vocab.stoi['language']

917

In [15]:
movie_reviews.vocab.itos[917]

'language'

In [16]:
c = Counter([4,2,8,8,4,8])
c

Counter({2: 1, 4: 2, 8: 3})

In [0]:
def get_term_doc_matrix(label_list, vocab_len):
  indices=[]
  indptr = []
  values = []
  indptr.append(0)
  
  for i, doc in enumerate(label_list):
    feature_counter = Counter(doc.data)
    indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    indptr.append(len(indices))
    
  return scipy.sparse.csr_matrix((values, indices, indptr),
                                 shape=(len(indptr) - 1, vocab_len),
                                 dtype=int
                                )

In [0]:
val_term_doc = get_term_doc_matrix(movie_reviews.valid.x, len(movie_reviews.vocab.itos))

In [0]:
trn_term_doc = get_term_doc_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))

In [20]:
trn_term_doc.shape, val_term_doc.shape

((800, 6008), (200, 6008))

In [21]:
movie_reviews.y.classes

['negative', 'positive']

In [0]:
x = trn_term_doc
y = movie_reviews.train.y
val_y = movie_reviews.valid.y

In [0]:
positive = y.c2i['positive']
negative = y.c2i['negative']

Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y.items.astype(int))
preds = m.predict(val_term_doc)
(preds == val_y.items).mean()

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.805

Naive Bayes

log-count ratio

$r = \log \frac{\text{ratio of feature $f$ in positive documents}}{\text{ratio of feature $f$ in negative documents}}$



In [0]:
p0=np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [0]:
p1=np.squeeze(np.asarray(x[y.items==positive].sum(0)))

In [0]:
pr1 = (p1 + 1)/ ((y.items==positive).sum() + 1)
pr0 = (p0 + 1)/ ((y.items==negative).sum() + 1)

In [0]:
r = np.log(pr1/pr0)

In [52]:
r[..., np.newaxis].shape, y.items.shape

((6008, 1), (800,))

In [36]:
(y.items==positive).mean(), (y.items==negative).mean()

(0.47875, 0.52125)

In [0]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean())

In [38]:
b

-0.08505123261815539

In [0]:
preds = (val_term_doc @ r + b) > 0

In [39]:
(preds == val_y.items).mean()


0.805